In [1]:
from agent import Agent


/Users/jackson.makl@dataiku.com/agent/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

a=Agent()


In [3]:

a.chat(prompt="What challenges does airbnb face?",
       web=False,
       rag=True,
       tokens=150)

/Users/jackson.makl@dataiku.com/agent/rag.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
100%|██████████| 4/4 [00:00<00:00, 1123.95it/s]


Loading model meta-llama/Llama-3.2-3B-Instruct on mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.23s/it]


business_&_entrepreneurs
news_&_social_concern


{'timestamp': '2025-04-28 01:47:43.751677',
 'topics': ['business_&_entrepreneurs', 'news_&_social_concern'],
 'web': False,
 'rag': True,
 'role': 'assistant',
 'content': "Airbnb faces several challenges:\n\n1. **Regulatory Issues**: The company has faced regulatory hurdles worldwide due to concerns over short-term rentals, tax evasion, and housing affordability.\n2. **Pricing Transparency**: Critics argue that Airbnb's algorithm-driven pricing can lead to discriminatory practices against certain groups or individuals, as seen in New York City where host demographics influence price disparities.\n\nTo address these concerns, I'd like to break down some key points from the context-provided discussion:\n\n*   Pricing Model Factors: When suggesting prices for listings, Airbnb uses various algorithms considering multiple parameters such as location (neighborhoods), property type, amenities offered by both guests & hosts, seasonal demand patterns, guest reviews ratings history among other

In [4]:
a.chat(prompt="What challenges does BMW face?",
       web=False,
       rag=True,
       tokens=150)
a.messages

100%|██████████| 4/4 [00:00<00:00, 1201.55it/s]


business_&_entrepreneurs
news_&_social_concern


[{'timestamp': '2025-04-28 01:47:43.751647',
  'topics': ['business_&_entrepreneurs', 'news_&_social_concern'],
  'web': False,
  'rag': True,
  'role': 'user',
  'content': 'What challenges does airbnb face?'},
 {'timestamp': '2025-04-28 01:47:43.751677',
  'topics': ['business_&_entrepreneurs', 'news_&_social_concern'],
  'web': False,
  'rag': True,
  'role': 'assistant',
  'content': "Airbnb faces several challenges:\n\n1. **Regulatory Issues**: The company has faced regulatory hurdles worldwide due to concerns over short-term rentals, tax evasion, and housing affordability.\n2. **Pricing Transparency**: Critics argue that Airbnb's algorithm-driven pricing can lead to discriminatory practices against certain groups or individuals, as seen in New York City where host demographics influence price disparities.\n\nTo address these concerns, I'd like to break down some key points from the context-provided discussion:\n\n*   Pricing Model Factors: When suggesting prices for listings, Air

In [5]:
a.chat(prompt="What is trending in music?",
       web=True,
       rag=False,
       tokens=50)
a.messages

Researching: What is trending in music?
News search error: cannot access local variable 'link' where it is not associated with a value
Found 10 total results
After deduplication: 10 results
Fetching additional content from URLs...
Researching: current trending songs and artists 2025 music charts TikTok viral tracks YouTube trends K-pop music news mental health in music
Found 15 total results
After deduplication: 11 results
Fetching additional content from URLs...
Error fetching content from https://duckduckgo.com/y.js?ad_domain=amazon.com&ad_provider=bingv7aa&ad_type=txad&click_metadata=QyLWBtu1f15pEJj4lFqU0MMPlEtEA4htXXreZ0UjllayuRGLd6c2ADEjF1Nr97Y3TV2x3Jpum6i0ERKtHk7lk78GbwHhg_VDLXfVvkX8H9VWaabTuPhvHuV5q6%2DgvA7y.kPv%2DhzZkd0d8nUAZD%2DBwAg&rut=b56a23b5b4a6207d0375a7204d08375174fa10d06b44d4a293630459966ad5d5&u3=https%3A%2F%2Fwww.bing.com%2Faclick%3Fld%3De8xLswpZhMI3UWh3KEBBTTqzVUCUz28RA%2DS7HX0mHbhi2u15YAGN1ZoJF1EqOQdDggCYACzcyW3oT8CmFcDOutyyEssH_3he4vuZHJe6nbqIBrFYZTBRqqkYpbKZ33chOoJ

[{'timestamp': '2025-04-28 01:47:43.751647',
  'topics': ['business_&_entrepreneurs', 'news_&_social_concern'],
  'web': False,
  'rag': True,
  'role': 'user',
  'content': 'What challenges does airbnb face?'},
 {'timestamp': '2025-04-28 01:47:43.751677',
  'topics': ['business_&_entrepreneurs', 'news_&_social_concern'],
  'web': False,
  'rag': True,
  'role': 'assistant',
  'content': "Airbnb faces several challenges:\n\n1. **Regulatory Issues**: The company has faced regulatory hurdles worldwide due to concerns over short-term rentals, tax evasion, and housing affordability.\n2. **Pricing Transparency**: Critics argue that Airbnb's algorithm-driven pricing can lead to discriminatory practices against certain groups or individuals, as seen in New York City where host demographics influence price disparities.\n\nTo address these concerns, I'd like to break down some key points from the context-provided discussion:\n\n*   Pricing Model Factors: When suggesting prices for listings, Air